In [ ]:
#| default_exp ctx

# CTX
> Utils for working with MRO CTX data

In [ ]:
#| hide
from nbdev.showdoc import show_doc

In [ ]:
#| export

import warnings
from pathlib import Path

import hvplot.xarray  # noqa
import rasterio
import rioxarray as rxr
from dask import compute, delayed
from planetarypy.config import config
from planetarypy.pds.apps import get_index
from planetarypy.utils import file_variations, url_retrieve
from tqdm.auto import tqdm
from yarl import URL
from fastcore.script import call_parse
from fastcore.basics import store_attr

try:
    from kalasiris.pysis import (
        ProcessError,
        ctxcal,
        ctxevenodd,
        getkey,
        mroctx2isis,
        spiceinit,
    )
except KeyError:
    warnings.warn("kalasiris has a problem initializing ISIS")

In [ ]:
#| export

warnings.filterwarnings("ignore", category=rasterio.errors.NotGeoreferencedWarning)
baseurl = URL(
    "https://pds-imaging.jpl.nasa.gov/data/mro/mars_reconnaissance_orbiter/ctx/"
)

storage_root = config.storage_root / "missions/mro/ctx"
edrindex = get_index("mro.ctx", "edr")

In [ ]:
edrindex.head()

,VOLUME_ID,FILE_SPECIFICATION_NAME,ORIGINAL_PRODUCT_ID,PRODUCT_ID,IMAGE_TIME,INSTRUMENT_ID,INSTRUMENT_MODE_ID,LINE_SAMPLES,LINES,SPATIAL_SUMMING,...,SUB_SOLAR_LATITUDE,SUB_SPACECRAFT_LONGITUDE,SUB_SPACECRAFT_LATITUDE,SOLAR_DISTANCE,SOLAR_LONGITUDE,LOCAL_TIME,IMAGE_SKEW_ANGLE,RATIONALE_DESC,DATA_QUALITY_DESC,ORBIT_NUMBER
0,MROX_0001,DATA/CRU_000001_9999_XN_99N999W.IMG,4A_04_0001000400,CRU_000001_9999_XN_99N999W,2005-08-30 15:40:21.549,CTX,NIFL,5056,1024,1,...,0.0,0.0,0.0,0.0,278.89,10.16,0.0,Instrument checkout image of space,OK,-4242
1,MROX_0001,DATA/CRU_000002_9999_XN_99N999W.IMG,4A_04_0001000500,CRU_000002_9999_XN_99N999W,2005-09-08 15:59:45.313,CTX,NIFL,5056,15360,1,...,0.0,0.0,0.0,0.0,284.48,4.60,0.0,Calibration image of the Moon,OK,-4126
2,MROX_0001,DATA/CRU_000003_9999_XN_99N999W.IMG,4A_04_0001000600,CRU_000003_9999_XN_99N999W,2005-09-08 16:03:37.927,CTX,NIFL,5056,2048,1,...,0.0,0.0,0.0,0.0,284.48,4.66,0.0,Calibration image of Omega Centauri (globular ...,OK,-4126
3,MROX_0001,DATA/CRU_000004_9999_XN_99N999W.IMG,4A_04_0001000700,CRU_000004_9999_XN_99N999W,2005-09-08 16:08:23.841,CTX,NIFL,5056,2048,1,...,0.0,0.0,0.0,0.0,284.48,4.74,0.0,Calibration image of Omega Centauri (globular ...,OK,-4126
4,MROX_0001,DATA/CRU_000005_9999_XN_99N999W.IMG,4A_04_0001000800,CRU_000005_9999_XN_99N999W,2005-09-08 16:11:18.649,CTX,NIFL,5056,21504,1,...,0.0,0.0,0.0,0.0,284.48,4.79,0.0,Calibration image of the Moon,OK,-4126


In [ ]:
#| export
def catch_isis_error(func):
    def inner(*args, **kwargs):
        try:
            func(*args, **kwargs)
        except ProcessError as err:
            print("Had ISIS error:")
            print(" ".join(err.cmd))
            print(err.stdout)
            print(err.stderr)

    return inner

In [ ]:
#| export
class CTXEDR:
    """Manage access to EDR data"""
    storage = storage_root / "edr"
    
    def __init__(
        self,
        id_:str,  # CTX product id (pid)
        source_dir:str='',  # alternative root folder for EDR data
        with_volume:bool=True,  # does the storage path include the volume folder
        with_id_dir:bool=False,  # does the storage path include an extra pid folder
    ):
        store_attr(but='source_dir')
        self.storage = Path(source_dir) if source_dir else self.storage
        
    @property
    def pid(self):
        return self.id_
    
    @pid.setter
    def pid(self, value):
        self.id_ = value
        
    @property
    def meta(self):
        "get the metadata from the index table"
        s = edrindex.query("PRODUCT_ID == @self.pid").squeeze()
        s.index = s.index.str.lower()
        return s

    @property
    def volume(self):
        "get the PDS volume number for the current product id"
        return self.meta.volume_id.lower()
    
    @property
    def source_folder(self):
        if self.with_volume:
            base = self.storage / self.volume
        else:
            base = self.storage
        if not self.with_id_dir:
            return base
        else: 
            return base / self.pid

    @property
    def source_path(self):
        return self.source_folder / f"{self.pid}.IMG"

    @property
    def url(self):
        "Calculate URL from input dataframe row."
        url = baseurl / self.meta.volume_id.lower() / "data" / (self.pid + ".IMG")
        return url

    def download(self, overwrite=False):
        if self.source_path.exists() and not overwrite:
            print("File exists. Use `overwrite=True` to download fresh.")
            return
        self.source_folder.mkdir(parents=True, exist_ok=True)
        url_retrieve(self.url, self.source_path)


In [ ]:
id_ = "F10_039666_1383_XN_41S315W"

In [ ]:
ctx = CTXEDR(id_)

In [ ]:
ctx.source_path

Path('/home/ayek72/mnt/slowdata/planetarypy/missions/mro/ctx/edr/mrox_2337/F10_039666_1383_XN_41S315W.IMG')

In [ ]:
ctx.meta

volume_id                                                 MROX_2337
file_specification_name         DATA/F10_039666_1383_XN_41S315W.IMG
original_product_id                                4A_04_10C800EF00
product_id                               F10_039666_1383_XN_41S315W
image_time                               2015-01-12 06:36:38.896000
instrument_id                                                   CTX
instrument_mode_id                                             NIFL
line_samples                                                   5056
lines                                                         15360
spatial_summing                                                   1
scaled_pixel_width                                             5.04
pixel_aspect_ratio                                              1.2
emission_angle                                                 1.29
incidence_angle                                                41.1
phase_angle                                     

In [ ]:
#| export
class CTX:
    """Class to manage dealing with CTX data.
    
    HAS a CTXEDR attribute as defined above.
    """
    proc_dir = storage_root / "edr"

    def __init__(
        self, 
        id_:str,  # CTX product id
        source_dir:str='',  # where the raw EDR data is stored, if not coming from plpy
        proc_dir:str='',  # where to store processed, if not plpy
        with_volume:bool=False,  # store with extra volume subfolder?
        with_id_dir:bool=True  # store with extra product_id subfolder?
    ):
        store_attr(but="source_dir,proc_dir")
        self.proc_dir = Path(proc_dir) if proc_dir else self.proc_dir
        self.edr = CTXEDR(id_, source_dir, with_volume, with_id_dir)
        
        (self.cub_name, self.cal_name, self.destripe_name) = file_variations(
            self.edr.source_path.name, [".cub", ".cal.cub", ".dst.cal.cub"]
        )
        self.is_read = False
        self.is_calib_read = False

    @property
    def pid(self):
        return self.edr.pid

    @property
    def proc_folder(self):
        "the folder for all processed data. could be same as source_dir"
        return self.proc_dir / self.edr.source_folder.relative_to(self.edr.source_folder)

    @property
    def cub_path(self):
        return self.proc_folder / self.cub_name
    
    @property
    def cal_path(self):
        return self.proc_folder / self.cal_name
    
    @property
    def destripe_path(self):
        return self.proc_folder / self.destripe_name


    @catch_isis_error
    def isis_import(self):
        mroctx2isis(from_=self.edr.source_path, to=self.cub_path)

    @catch_isis_error
    def spice_init(self):
        spiceinit(from_=self.cub_path, web="yes")

    @catch_isis_error
    def calibrate(self):
        ctxcal(from_=self.cub_path, to=self.cal_path)
        self.is_calib_read = False

    @catch_isis_error
    def destripe(self):
        if self.do_destripe():
            ctxevenodd(from_=self.cal_path, to=self.destripe_path)
            self.destripe_path.rename(self.cal_path)

    @catch_isis_error
    def do_destripe(self):
        value = int(
            getkey(
                from_=self.cal_path,
                objname="isiscube",
                grpname="instrument",
                keyword="SpatialSumming",
            )
        )
        return False if value == 2 else True

    def calib_pipeline(self, overwrite=False):
        if self.cal_path.exists() and not overwrite:
            return
        pbar = tqdm("isis_import spice_init calibrate destripe".split())
        for name in pbar:
            pbar.set_description(name)
            getattr(self, name)()
        pbar.set_description("Done.")

    def read_edr(self):
        "`da` stands for dataarray, standard abbr. within xarray."
        if not self.edr.source_path.exists():
            raise FileNotFoundError("EDR not downloaded yet.")
        if not self.is_read:
            self.edr_da = rxr.open_rasterio(self.edr.source_path)
            self.is_read = True
        return self.edr_da

    def read_calibrated(self):
        "`da` stands for dataarray, standard abbr. within xarray."
        if not self.is_calib_read:
            self.cal_da = rxr.open_rasterio(self.cal_path)
            self.is_calibd_read = True
        return self.cal_da

    def plot_da(self, data=None):
        data = self.edr_da if data is None else data
        return data.isel(band=0, drop=True).hvplot(
            x="y", y="x", rasterize=True, cmap="gray", data_aspect=1
        )

    def plot_calibrated(self):
        return self.plot_da(self.read_calibrated())

    def __str__(self):
        s = f"PRODUCT_ID: {self.edr.pid}\n"
        s += f"URL: {self.edr.url}\n"
        s += f"Local: {self.edr.source_path}\n"
        try:
            s += f"Shape: {self.read_edr().shape}"
        except FileNotFoundError:
            s += f"Not downloaded yet."
        return s

    def __repr__(self):
        return self.__str__()

In [ ]:
id_ = "F10_039666_1383_XN_41S315W"

In [ ]:
ctx = CTX(id_)

In [ ]:
ctx

PRODUCT_ID: F10_039666_1383_XN_41S315W
URL: https://pds-imaging.jpl.nasa.gov/data/mro/mars_reconnaissance_orbiter/ctx/mrox_2337/data/F10_039666_1383_XN_41S315W.IMG
Local: /home/ayek72/mnt/slowdata/planetarypy/missions/mro/ctx/edr/F10_039666_1383_XN_41S315W/F10_039666_1383_XN_41S315W.IMG
Shape: (1, 15360, 5056)

In [ ]:
ctx.pid

'F10_039666_1383_XN_41S315W'

In [ ]:
ctx.edr.meta

volume_id                                                 MROX_2337
file_specification_name         DATA/F10_039666_1383_XN_41S315W.IMG
original_product_id                                4A_04_10C800EF00
product_id                               F10_039666_1383_XN_41S315W
image_time                               2015-01-12 06:36:38.896000
instrument_id                                                   CTX
instrument_mode_id                                             NIFL
line_samples                                                   5056
lines                                                         15360
spatial_summing                                                   1
scaled_pixel_width                                             5.04
pixel_aspect_ratio                                              1.2
emission_angle                                                 1.29
incidence_angle                                                41.1
phase_angle                                     

In [ ]:
ctx.edr.source_folder

Path('/home/ayek72/mnt/slowdata/planetarypy/missions/mro/ctx/edr/F10_039666_1383_XN_41S315W')

In [ ]:
ctx.proc_folder

Path('/home/ayek72/mnt/slowdata/planetarypy/missions/mro/ctx/edr')

In [ ]:
ctx.edr.volume

'mrox_2337'

In [ ]:
CTXEDR(id_, with_volume=True, with_id_dir=True).source_folder

Path('/home/ayek72/mnt/slowdata/planetarypy/missions/mro/ctx/edr/mrox_2337/F10_039666_1383_XN_41S315W')

In [ ]:
assert str(CTXEDR(id_, with_volume=True).source_folder) ==\
'/home/ayek72/mnt/slowdata/planetarypy/missions/mro/ctx/edr/mrox_2337'

In [ ]:
assert str(CTXEDR(id_, with_volume=True, with_id_dir=True).source_folder) ==\
'/home/ayek72/mnt/slowdata/planetarypy/missions/mro/ctx/edr/mrox_2337/F10_039666_1383_XN_41S315W'

In [ ]:
ctx.edr.url

URL('https://pds-imaging.jpl.nasa.gov/data/mro/mars_reconnaissance_orbiter/ctx/mrox_2337/data/F10_039666_1383_XN_41S315W.IMG')

In [ ]:
ctx.edr.download()

File exists. Use `overwrite=True` to download fresh.


In [ ]:
show_doc(CTX.calib_pipeline)

---

### CTX.calib_pipeline

>      CTX.calib_pipeline (overwrite=False)

In [ ]:
ctx.isis_import()

In [ ]:
ctx.spice_init()

In [ ]:
ctx.calibrate()

In [ ]:
ctx.destripe()

In [ ]:
ctx.calib_pipeline()

In [ ]:
ctx.plot_calibrated()

:DynamicMap   []
   :Image   [y,x]   (value)

In [ ]:
ds = ctx.read_edr()
ds

<xarray.DataArray (band: 1, y: 15360, x: 5056)>
[77660160 values with dtype=uint8]
Coordinates:
  * band         (band) int64 1
  * x            (x) float64 0.5 1.5 2.5 3.5 ... 5.054e+03 5.054e+03 5.056e+03
  * y            (y) float64 0.5 1.5 2.5 3.5 ... 1.536e+04 1.536e+04 1.536e+04
    spatial_ref  int64 0
Attributes:
    _FillValue:    0.0
    scale_factor:  1.0
    add_offset:    0.0

## CLI for calibrating single CTX file

In [ ]:
#| export
@call_parse
def ctx_calib(
    id_:str,  # CTX product_id
    source:str='',  # path to where EDRs are stored if not from plpy
    proc_dir:str='',  # path to where processed data is to be stored
    overwrite:bool=False,  # overwrite processed data
):
    ctx = CTX(id_, source_dir=source, proc_dir=proc_dir)
    ctx.calib_pipeline(overwrite=overwrite)
    print("Produced\n", ctx.cal_path)

In [ ]:
ctx_calib(id_, overwrite=True)

  0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
#| export

class CTXEDRCollection:
    """Class to deal with a set of CTX products."""

    def __init__(self, product_ids):
        self.product_ids = product_ids

    def get_urls(self):
        """Get URLs for list of product_ids.

        Returns
        -------
        List[yarl.URL]
            List of URL objects with the respective PDS URL for download.
        """
        urls = []
        for p_id in self.product_ids:
            ctx = CTXEDR(p_id)
            urls.append(ctx.url)
        self.urls = urls
        return urls

    def download_collection(self):
        lazys = []
        for p_id in self.product_ids:
            ctx = CTXEDR(p_id)
            lazys.append(delayed(ctx.download)())
        print("Launching parallel download...")
        compute(*lazys)
        print("Done.")

    def calibrate_collection(self):
        lazys = []
        for p_id in self.product_ids:
            ctx = CTXEDR(p_id)
            lazys.append(delayed(ctx.calib_pipeline)())
        print("Launching parallel calibration...")
        compute(*lazys)
        print("Done.")

    def calib_exist_check(self):
        return [(p_id, CTXEDR(p_id).cal_name.exists()) for p_id in self.product_ids]

In [ ]:
ids = edrindex.sample(3, random_state=42).PRODUCT_ID
ids

33054     G01_018691_2639_XN_83N125W
93213     J18_051799_1768_XN_03S008W
116150    N07_065025_2026_XN_22N001W
Name: PRODUCT_ID, dtype: object

In [ ]:
coll = CTXEDRCollection(ids)

In [ ]:
coll.get_urls()

[URL('https://pds-imaging.jpl.nasa.gov/data/mro/mars_reconnaissance_orbiter/ctx/mrox_1189/data/G01_018691_2639_XN_83N125W.IMG'),
 URL('https://pds-imaging.jpl.nasa.gov/data/mro/mars_reconnaissance_orbiter/ctx/mrox_2936/data/J18_051799_1768_XN_03S008W.IMG'),
 URL('https://pds-imaging.jpl.nasa.gov/data/mro/mars_reconnaissance_orbiter/ctx/mrox_3682/data/N07_065025_2026_XN_22N001W.IMG')]

The next command launches a parallel download, conveniently:

In [ ]:
lazys = coll.download_collection()
lazys

Launching parallel download...


G01_018691_2639_XN_83N125W.IMG
:   0%|          | 0/36246464 [00:00<?, ?it/s]

N07_065025_2026_XN_22N001W.IMG
:   0%|          | 0/264049600 [00:00<?, ?it/s]

J18_051799_1768_XN_03S008W.IMG
:   0%|          | 0/36246464 [00:00<?, ?it/s]

Done.
